In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

gs


In [14]:
# poetry add pydantic
# %pip install pydantic 

from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

In [16]:
# 출력 구조를 정의하는 Pydantic 모델
class MovieRecommendation(BaseModel):
    name: str = Field(description="이름")
    age: int = Field(description="나이")
    major: str = Field(description="전공")
    hobbies: List[str] = Field(description="취미 리스트")
    goal: str = Field(description="목표")
    
# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=MovieRecommendation)

# 프롬프트 템플릿 설정
template = """
다음 사용자 요청에 따라 자기소개를 받아 출력해주세요.
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)
print(prompt)

input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"description": "이름", "title": "Name", "type": "string"}, "age": {"description": "나이", "title": "Age", "type": "integer"}, "major": {"description": "전공", "title": "Major", "type": "string"}, "hobbies": {"description": "취미 리스트", "items": {"type": "string"}, "title": "Hobbies", "type": "array"}, "goal": {"description": "목표", "title": "Goal", "type": "string"}}, "required": ["name", "age", "major", "hobbies", "goal"]}\n```'} messages

In [19]:

# ChatOpenAI 모델 초기화
# model = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo")
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 구성 및 실행
query = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
chain = prompt | model | parser
output = chain.invoke({"query": query})

# 결과 출력
print(f"name: {output.name}")
print(f"age: {output.age}")
print(f"major: {(output.major)}")
print(f"hobbies: {', '.join(output.hobbies)}")
print(f"목표: {output.goal}")

name: 김민수
age: 22
major: 컴퓨터공학
hobbies: 게임하기, 영화보기, 코딩
목표: 훌륭한 개발자가 되는 것
